In [ ]:
  pip install transformers datasets torch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from datasets import load_dataset

# "OsBaran/try" veri setini yükleyin
dataset = load_dataset("OsBaran/try")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/358 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/94.3M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/78588 [00:00<?, ? examples/s]

In [ ]:
print(dataset)  # Tüm veri kümesini yazdırır
print(dataset['train'][0])

DatasetDict({
    train: Dataset({
        features: ['text', 'label', '__index_level_0__'],
        num_rows: 78588
    })
})
{'text': 'head conservative republican faction us congress, voted month huge expansion national debt pay tax cuts, called fiscal conservative sunday urged budget restraint keeping sharp pivot way among republicans, us representative mark meadows, speaking cbs face nation, drew hard line federal spending, lawmakers bracing battle january return holidays wednesday, lawmakers begin trying pass federal budget fight likely linked issues, immigration policy, even november congressional election campaigns approach republicans seek keep control congress president donald trump republicans want big budget increase military spending, democrats also want proportional increases non defense discretionary spending programs support education, scientific research, infrastructure, public health environmental protection trump administration already willing say going increase non 

In [ ]:
from sklearn.model_selection import train_test_split

import pandas as pd

# Dataset'i pandas DataFrame'e dönüştür
df = pd.DataFrame(dataset['train'])

# Eğitim ve test setlerini oluştur
train_data, test_data = train_test_split(df, test_size=0.2)

# Metinleri ve etiketleri ayırın
train_texts = train_data['text']
train_labels = train_data['label']
test_texts = test_data['text']
test_labels = test_data['label']

In [ ]:
from transformers import RobertaTokenizer

# Tokenizer'ı yükleyin
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

# Metinleri tokenleştirin
train_encodings = tokenizer(train_texts.tolist(), truncation=True, padding=True, max_length=512)
test_encodings = tokenizer(test_texts.tolist(), truncation=True, padding=True, max_length=512)

In [ ]:
import torch
class NewsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = NewsDataset(train_encodings, train_labels.tolist())
test_dataset = NewsDataset(test_encodings, test_labels.tolist())

In [ ]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # Cihazı kontrol et
model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=2).to(device)  # Modeli cihaza yükle

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from huggingface_hub import login
login(token="")

In [ ]:
training_args = TrainingArguments(
    num_train_epochs=5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    gradient_accumulation_steps=4,
    learning_rate=2e-5,
    fp16=True,
    warmup_steps=500,
    weight_decay=0.01,  # Ağırlık çürümesi
    evaluation_strategy="epoch",
    save_steps=200,
    logging_steps=10,
    push_to_hub=True,
    output_dir="Roberta-Classification-Model",
    optim="adamw_torch"
)

NameError: name 'TrainingArguments' is not defined

In [ ]:
from transformers import DataCollatorWithPadding
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="pt")
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [ ]:
trainer.train()


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss
0,0.075700,0.083351
2,0.040700,0.060591


KeyboardInterrupt: 

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Roberta model ve tokenizer'ı yükle
tokenizer = AutoTokenizer.from_pretrained("roberta-base")
model = AutoModelForSequenceClassification.from_pretrained("OsBaran/Roberta-Classification-Model",output_attentions=True,revision="769765b6ca3af94b693915f1562d6f7010215c70")

# Modeli GPU'ya taşıyın (eğer varsa)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/735 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

In [ ]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments
training_args = TrainingArguments(
    num_train_epochs=5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=4,
    learning_rate=2e-5,
    fp16=True,
    warmup_steps=500,
    weight_decay=0.01,  # Ağırlık çürümesi
    evaluation_strategy="epoch",
    save_steps=200,
    logging_steps=10,
    push_to_hub=True,
    output_dir="Roberta-Classification-Model",
    optim="adamw_torch"
)
from transformers import DataCollatorWithPadding
trainer = Trainer(
    model=model,
    args=training_args,
    # train_dataset=train_dataset,
    train_dataset=dataset,

    eval_dataset=dataset,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="pt")
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
def predict_with_roberta(model, tokenizer, input_text):
    # Giriş metnini tokenize et ve tensor'a çevir
    inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True).to(device)

    # Model ile tahmin yap
    with torch.no_grad():
        outputs = model(**inputs)

    # Logits'leri al ve tahmin yap
    logits = outputs.logits
    prediction = torch.argmax(logits, dim=-1).item()  # 0: yanlış, 1: doğru
    return prediction

In [ ]:
def explain_roberta_prediction(model, tokenizer, input_text):
    # Tokenize et

    inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True).to(device)
    # Model ile tahmin yap
    with torch.no_grad():
        outputs = model(**inputs)

    # Logits'leri al
    logits = outputs.logits
    probabilities = torch.softmax(logits, dim=-1).cpu().numpy()[0]

    # Tahmin sonucunu ve olasılıkları elde et
    predicted_class = torch.argmax(logits, dim=-1).item()
    result = "Doğru" if predicted_class == 1 else "Yanlış"
    explanation = f"Modelin tahmini: {result} (Olasılık: {probabilities[predicted_class]:.2f})\n"

    # Önemli kelimeleri çıkarma (örnek olarak)
    tokenized_input = tokenizer.tokenize(tokenizer.decode(inputs['input_ids'][0]))
    important_tokens = tokenized_input[:10]  # İlk 10 tokeni al
    explanation += "Modelin kararı aşağıdaki anahtar kelimelere dayanıyor:\n" + ', '.join(important_tokens)

    return explanation


In [ ]:
!pip install shap

In [ ]:
import shap
from transformers import (AutoTokenizer,
                          AutoModelForSequenceClassification,
                          TextClassificationPipeline)
pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer, return_all_scores=True, device=device)
def score_and_visualize(text):
  prediction = pipe([text])
  print(prediction[0])

  explainer = shap.Explainer(pipe)
  shap_values = explainer([text])
  shap.plots.text(shap_values)


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [ ]:
import requests
import re
from collections import Counter
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
api_key = '764e3b45715b449a8aedb8cd8018dfed'
def fetch_news_from_api(api_key, query, page_size=100):
    url = f'https://newsapi.org/v2/everything?q={query}&pageSize={page_size}&apiKey={api_key}'
    response = requests.get(url)

    # API yanıtını kontrol et
    if response.status_code == 200:
        articles = response.json().get('articles', [])
        return articles
    else:
        print(f"Error: {response}")
        return []

In [ ]:
def extract_keywords(text, top_n=5):
    # 1. Metni temizleme
    text = re.sub(r'[^\w\s]', '', text.lower())  # Noktalama işaretlerini kaldırma ve küçük harfe çevirme

    # 2. Tokenizasyon
    words = text.split()

    # 3. Durak kelimeleri kaldırma
    keywords = [word for word in words if word not in ENGLISH_STOP_WORDS]

    # 4. Anahtar kelimeleri sayma ve en sık geçenleri alma
    keyword_counts = Counter(keywords)
    most_common_keywords = keyword_counts.most_common(top_n)

    return [keyword for keyword, _ in most_common_keywords]

In [ ]:
!pip install keybert
!pip install sentence-transformers

In [ ]:
from keybert import KeyBERT

kw_model = KeyBERT('all-mpnet-base-v2')  # SBERT kullanarak modeli yükleyin

def extract_keywords_keybert(text, num_keywords=2):
    keywords = kw_model.extract_keywords(text, keyphrase_ngram_range=(1, 2), stop_words='english', top_n=num_keywords)
    return [kw[0] for kw in keywords]



modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:

def filter_trusted_sources(articles, trusted_sources):
    trusted_articles = []
    for article in articles:
        source_name = article['source']['name'].lower()  # Kaynağı küçük harfe çevir
        if any(trusted_source in source_name for trusted_source in trusted_sources):
          trusted_articles.append(article)
    return trusted_articles

In [ ]:

def fetch_news_content(link):
    response = requests.get(link)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        # Başlık ve içerik çıkarma
        title = soup.find('h1').get_text() if soup.find('h1') else "Başlık bulunamadı"
        content = ' '.join([p.get_text() for p in soup.find_all('p')])
        return title, content
    else:
        print(f"Error fetching content: {response.status_code}")
        return "", "";

In [ ]:
def compare_with_thrusted(input_text, bbc_articles):
    texts = [input_text] + [article[1] for article in bbc_articles]
    vectorizer = TfidfVectorizer().fit_transform(texts)
    vectors = vectorizer.toarray()
    similarities = cosine_similarity(vectors[0:1], vectors[1:]).flatten()
    return similarities


In [ ]:
from sentence_transformers import SentenceTransformer, util
def sbert_similarity(input_text, bbc_articles):
# SBERT modelini yükleyin
    model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

    # Kullanıcı metni ve internetten çekilen metinleri vektörize edin
    input_embedding = model.encode(input_text, convert_to_tensor=True)
    news_embeddings = model.encode([news[1] for news in bbc_articles], convert_to_tensor=True)

    # Benzerlikleri hesaplayın
    cosine_scores = util.pytorch_cos_sim(input_embedding, news_embeddings)

    # En yüksek benzerlik skoru ve karşılık gelen haber
    max_score, most_similar_news = cosine_scores.max(), bbc_articles[cosine_scores.argmax().item()]

    return max_score



In [ ]:
!pip install deep-translator

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 1.5 MB/s eta 0:00:00


In [ ]:
from deep_translator import DeeplTranslator

# DeepL API anahtarınızı buraya girin
api_key = ""

def translate_text(text, source_lang='tr', target_lang='en'):
    translated = DeeplTranslator(api_key=api_key, source=source_lang, target=target_lang).translate(text)
    return translated

# Örnek kullanım


In [ ]:
!pip install vadersentiment

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 1.0 MB/s eta 0:00:00


In [ ]:
import requests
from bs4 import BeautifulSoup

# Bağlantıdan içerik çekme fonksiyonu
def fetch_content_from_url(url):
    try:
        response = requests.get(url, timeout=10)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            # Genellikle <p> etiketleri içerik için kullanılır
            paragraphs = soup.find_all('p')
            content = " ".join([para.get_text() for para in paragraphs])
            return content
        else:
            print(f"Bağlantı başarısız: {url}")
            return None
    except Exception as e:
        print(f"URL'den içerik alınamadı: {url}, Hata: {e}")
        return None

def generate_report_with_content(verification_result):
    if verification_result["status"] == "Verified":
        result = verification_result["result"]
        report = f"""
        ✅ Doğrulama Başarılı!
        - Başlık: {result['title']}
        - Link: {result['link']}
        - Snippet Benzerlik Skoru: {result['snippet_similarity']:.2f}
        - İçerik Benzerlik Skoru: {result['content_similarity']:.2f}
        - Güvenilir Kaynak: {'Evet' if result['is_trusted'] else 'Hayır'}
        - Duygu Analizi (İçerik): {result['sentiment_content']}
        """
    else:
        report = f"""
        ⚠️ Bilgi Şüpheli!
        - Mesaj: {verification_result['message']}
        """
    print(report)
    return report

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# Duygu analiz fonksiyonuz
def analyze_sentiment(text):
    analyzer = SentimentIntensityAnalyzer()
    sentiment_score = analyzer.polarity_scores(text)
    return sentiment_score
def evaluate_results_with_content(scores):
    for result in scores:
        # İçerik ve snippet benzerliği ile duygu değerlendirmesi
        if result['content_similarity'] > 0.6 or result['snippet_similarity'] > 0.6:
            if result['is_trusted'] and result['sentiment_content']['compound'] > -0.1:
                print(f"Doğrulandı: {result['title']} ({result['link']})")
                return {"status": "Verified", "result": result}
    return {"status": "Suspicious", "message": "Hiçbir sonuç yeterince güvenilir görünmüyor."}

from sentence_transformers import SentenceTransformer, util

# Sentence Transformer modelini yükle
model_sentence = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Gelişmiş benzerlik hesaplama
def advanced_similarity(text1, text2):
    embeddings1 = model_sentence.encode(text1, convert_to_tensor=True)
    embeddings2 = model_sentence.encode(text2, convert_to_tensor=True)
    cosine_sim = util.cos_sim(embeddings1, embeddings2)
    return cosine_sim.item()
def search_google(query, num_results=10):
    search_results = []
    start = 1
    while len(search_results) < num_results:
        url = f'https://www.googleapis.com/customsearch/v1?q={query}&key={API_KEY}&cx={CX}&start={start}'
        response = requests.get(url)
        results = response.json()
        if 'items' in results:
            search_results.extend(results['items'])
            start += 10  # Bir sonraki sayfaya geç
        else:
            break
    return search_results[:num_results]
# Güvenilir kaynak listesi
TRUSTED_SOURCES = ['bbc.com', 'cnn.com', 'reuters.com', 'theguardian.com']

# Kaynağı kontrol etme
def check_source_reliability(link):
    for source in TRUSTED_SOURCES:
        if source in link:
            return True
    return False
def verify_information_with_content(input_text):
    print(f"'{input_text}' bilgisi için arama yapılıyor...\n")
    results = search_google(input_text)

    if not results:
        print("Hiçbir sonuç bulunamadı. Bilgi doğrulanamıyor.")
        return {"status": "No Results", "message": "Bilgiyle ilgili arama sonuçları bulunamadı."}

    scores = []
    for result in results:
        print(result)
        similarity_snippet = 0
        similarity_snippet = advanced_similarity(input_text, result['snippet'])
        is_trusted = check_source_reliability(result['link'])

        # Bağlantı içeriğini al ve analiz et
        content = fetch_content_from_url(result['link'])
        if content:
            similarity_content = advanced_similarity(input_text, content)
            sentiment_content = analyze_sentiment(content)
        else:
            similarity_content = 0
            sentiment_content = {"compound": 0}

        scores.append({
            "title": result['title'],
            "link": result['link'],
            "snippet": result['snippet'],
            "content_similarity": similarity_content,
            "snippet_similarity": similarity_snippet,
            "is_trusted": is_trusted,
            "sentiment_content": sentiment_content
        })

        print(f"Başlık: {result['title']}")
        print(f"Snippet Benzerlik Skoru: {similarity_snippet:.2f}")
        print(f"İçerik Benzerlik Skoru: {similarity_content:.2f}")
        print(f"Güvenilir Kaynak: {is_trusted}")
        print(f"Duygu Analizi (İçerik): {sentiment_content}\n")

    return evaluate_results_with_content(scores)

# Örnek Kullanım


# Örnek Kullanım


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
!pip install transformers matplotlib


In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch
import matplotlib.pyplot as plt
import numpy as np
input_lang = input("Dil seçiniz!")
input_news = input("Haber metnini girin: ")
if(input_lang=="tr"):
  input_news= translate_text(input_news)
# Kullanıcıdan haber metni al
# inputs = tokenizer(input_news, return_tensors="pt")

# # Modeli çalıştır
# outputs = model(**inputs)
# attention = outputs.attentions  # Tüm dikkat skorları burada

# # İlk katman ve ilk başlık için dikkat haritasını seç
# attention_layer = attention[0]  # İlk katman
# attention_head = attention_layer[0][0]  # İlk başlık için [batch, head, seq_len, seq_len]

# # Dikkat skorlarını görselleştir
# tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])

# plt.figure(figsize=(10, 8))
# plt.imshow(attention_head.detach().numpy(), cmap="viridis")
# plt.xticks(ticks=np.arange(len(tokens)), labels=tokens, rotation=90)
# plt.yticks(ticks=np.arange(len(tokens)), labels=tokens)
# plt.colorbar(label="Attention Weight")
# plt.title("Attention Map (Layer 1, Head 1)")
# plt.show()

# score_and_visualize(input_news)
# verification_result = verify_information_with_content(input_news)
# generate_report_with_content(verification_result)




keywords = extract_keywords_keybert(input_news)
print("KeyBERT Anahtar Kelimeler:", keywords)

# API çağrısı için anahtar kelimeleri kullanma
# Anahtar kelimeleri birleştirerek arama terimi oluşturma
search_query = ' '.join(keywords)
news_articles = fetch_news_from_api(api_key, search_query)

trusted_sources = [
    "bbc news",
    "cnn",
    "reuters.com",
    "theguardian.com",
    "time",
    # Diğer güvenilir kaynaklar...
]

trusted_articles = filter_trusted_sources(news_articles, trusted_sources)
# # Sonuçları yazdır
trusted_articles_urls = []
for i in trusted_articles:
    trusted_articles_urls.append(i["url"])

if trusted_articles:
    print(f"\nGüvenilir kaynaklardan bulunan haberler:\n")
    print(trusted_articles_urls)
    bbc_articles = [fetch_news_content(link) for link in trusted_articles_urls]
    # similarities = compare_with_thrusted(input_news, bbc_articles)
    max_similarity = sbert_similarity(input_news, bbc_articles)
    print(f"En benzer haber skoru: {max_similarity:.2f}")
    # print(similarities)
    # max_similarity = max(similarities)
    threshold = 1.2
    if max_similarity > threshold:
        print(f"Sonuç: Doğru (Benzerlik: {max_similarity:.2f})")
    else:
        # Benzerlik bulunmazsa tahmin algoritmasını kullanın ve açıklama sağlayın
        prediction = predict_with_roberta(model, tokenizer, input_news)
        print(prediction)
        explanation = explain_roberta_prediction(model, tokenizer, input_news)
        # Tahmin sonucunu yazdır
        # result = "Doğru" if prediction == 1 else "Yanlış"
        # print(f"Haberin durumu: {result}")
        print(explanation)




else:
    print("Güvenilir kaynaklardan hiç haber bulunamadı.")
    prediction = predict_with_roberta(model, tokenizer, input_news)
    explanation = explain_roberta_prediction(model, tokenizer, input_news)
    # Tahmin sonucunu yazdır
    result = "Doğru" if prediction == 1 else "Yanlış"
    print(f"Haberin durumu: {result}")
    print("Haberin açıklaması:")
    print(explanation)


Dil seçiniz!
Haber metnini girin: Forty years on from the original recording, the cream of British and Irish pop music past and present are once again asking whether Ethiopians know it is Christmas.  In 1984, responding to horrific images of the famine in northern Ethiopia broadcast on the BBC, musicians Bob Geldof and Midge Ure corralled some of the biggest stars of the era to record a charity song.  The release of the Band Aid single, and the Live Aid concert that followed eight months later, became seminal moments in celebrity fundraising and set a template that many others followed.  Do They Know It’s Christmas? is back on Monday with a fresh mix of the four versions of the song that have been issued over the years.  But the chorus of disapproval about the track, its stereotypical representation of an entire continent - describing it as a place "where nothing ever grows; no rain nor rivers flow" - and the way that recipients of the aid have been viewed as emaciated, helpless figure

In [ ]:
eval_predictions = trainer.predict(test_dataset)

# Tahmin sonuçlarını al
predictions = eval_predictions.predictions
labels = eval_predictions.label_ids

# Doğruluk, precision, recall vb. hesaplamak için
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

accuracy = accuracy_score(labels, predictions.argmax(axis=1))
precision = precision_score(labels, predictions.argmax(axis=1))
recall = recall_score(labels, predictions.argmax(axis=1))
f1 = f1_score(labels, predictions.argmax(axis=1))

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Accuracy: 0.9946
Precision: 0.9923
Recall: 0.9955
F1 Score: 0.9939


In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("bhavikjikadara/fake-news-detection")

print("Path to dataset files:", path)

100%|██████████| 41.0M/41.0M [00:00<00:00, 67.8MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/bhavikjikadara/fake-news-detection/versions/1


In [ ]:
import pandas as pd


fake_df = pd.read_csv('/root/.cache/kagglehub/datasets/bhavikjikadara/fake-news-detection/versions/1/fake.csv')  # 'fake.csv' dosyasını yükleyin
true_df = pd.read_csv('/root/.cache/kagglehub/datasets/bhavikjikadara/fake-news-detection/versions/1/true.csv')  # 'true.csv' dosyasını yükleyin

# Etiketleri belirleyin: Fake haberler 0, True haberler 1
fake_df['label'] = 0  # Fake haberler için etiket 0
true_df['label'] = 1  # True haberler için etiket 1

# Verileri birleştirelim

df = pd.concat([fake_df[['text', 'label']], true_df[['text', 'label']]], ignore_index=True)
# İlk birkaç satırı kontrol edelim
print(df.head())


                                                text  label
0  Donald Trump just couldn t wish all Americans ...      0
1  House Intelligence Committee Chairman Devin Nu...      0
2  On Friday, it was revealed that former Milwauk...      0
3  On Christmas day, Donald Trump announced that ...      0
4  Pope Francis used his annual Christmas Day mes...      0


In [ ]:
inputs = tokenizer(df['text'].tolist(), padding=True, truncation=True, return_tensors="pt", max_length=512).to(device)

# Etiketleri ayıralım
labels = torch.tensor(df['label'].tolist())
import torch
class NewsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

test_dataset2 = NewsDataset(inputs, labels)

In [ ]:
eval_predictions = trainer.predict(test_dataset2)

# Tahmin sonuçlarını al
predictions = eval_predictions.predictions
labels = eval_predictions.label_ids

# Doğruluk, precision, recall vb. hesaplamak için
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

accuracy = accuracy_score(labels, predictions.argmax(axis=1))
precision = precision_score(labels, predictions.argmax(axis=1))
recall = recall_score(labels, predictions.argmax(axis=1))
f1 = f1_score(labels, predictions.argmax(axis=1))

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

<ipython-input-116-3865bd09705c>:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-116-3865bd09705c>:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item['labels'] = torch.tensor(self.labels[idx])


Accuracy: 0.9567
Precision: 0.9172
Recall: 0.9994
F1 Score: 0.9565


In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("saurabhshahane/fake-news-classification")

print("Path to dataset files:", path)

100%|██████████| 92.1M/92.1M [00:02<00:00, 43.2MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/saurabhshahane/fake-news-classification/versions/77


In [ ]:
import pandas as pd


test3_df = pd.read_csv('/root/.cache/kagglehub/datasets/saurabhshahane/fake-news-classification/versions/77/WELFake_Dataset.csv')  # 'fake.csv' dosyasını yükleyin
# Gereksiz sütunları kaldırma
test3_df = test3_df[['text', 'label']].dropna()

# Metin verisini ve etiketleri ayırma
texts = test3_df['text'].tolist()
labels = test3_df['label'].tolist()


In [ ]:
inputs = tokenizer(test3_df['text'].tolist(), padding=True, truncation=True, return_tensors="pt", max_length=512).to(device)

# Etiketleri ayıralım
labels = torch.tensor(test3_df['label'].tolist())
import torch
class NewsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

test_dataset3 = NewsDataset(inputs, labels)

In [ ]:
eval_predictions = trainer.predict(test_dataset3)

# Tahmin sonuçlarını al
predictions = eval_predictions.predictions
labels = eval_predictions.label_ids

# Doğruluk, precision, recall vb. hesaplamak için
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

accuracy = accuracy_score(labels, predictions.argmax(axis=1))
precision = precision_score(labels, predictions.argmax(axis=1))
recall = recall_score(labels, predictions.argmax(axis=1))
f1 = f1_score(labels, predictions.argmax(axis=1))

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

NameError: name 'trainer' is not defined

In [ ]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments
training_args = TrainingArguments(
    num_train_epochs=5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=16,
    gradient_accumulation_steps=4,
    learning_rate=2e-5,
    fp16=True,
    warmup_steps=500,
    weight_decay=0.01,  # Ağırlık çürümesi

    save_steps=200,
    logging_steps=10,
    push_to_hub=True,
    output_dir="Roberta-Classification-Model",
    optim="adamw_torch"
)
from transformers import DataCollatorWithPadding
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=test_dataset3,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="pt")
)

In [ ]:
trainer.train()

<ipython-input-5-a3cc1ae35d1f>:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-5-a3cc1ae35d1f>:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item['labels'] = torch.tensor(self.labels[idx])


Step,Training Loss
10,3.018800
20,2.817200
30,2.389900
40,1.783600
50,1.160600
60,0.805800
70,0.666200
80,0.544500
90,0.379200
100,0.236900


<ipython-input-5-a3cc1ae35d1f>:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-5-a3cc1ae35d1f>:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item['labels'] = torch.tensor(self.labels[idx])
<ipython-input-5-a3cc1ae35d1f>:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-5-a3cc1ae35d1f>:13: UserWarning: To copy construct from a tensor, it is recommended to

KeyboardInterrupt: 

In [ ]:
from datasets import load_dataset
dataset = load_dataset("OsBaran/isot_news_dataset")

README.md:   0%|          | 0.00/313 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/68.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/44898 [00:00<?, ? examples/s]

In [ ]:
tokenized_test = dataset['train'].map(
    lambda x: tokenizer(x['text'], padding="max_length", truncation=True),
    batched=True
)

Map:   0%|          | 0/44898 [00:00<?, ? examples/s]

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import numpy as np
import matplotlib.pyplot as plt
trainer.model.to("cpu")

subset = tokenized_test.select(range(100))

 # İlk 1000 örneği seç
predictions = trainer.predict(subset)

y_pred = np.argmax(predictions.predictions, axis=1)
y_true = predictions.label_ids

cm = confusion_matrix(y_true, y_pred)

# Çizim
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=model.config.id2label.values())
disp.plot(cmap="Blues")
plt.title("Confusion Matrix")
plt.show()

RobertaSdpaSelfAttention is used but `torch.nn.functional.scaled_dot_product_attention` does not support non-absolute `position_embedding_type` or `output_attentions=True` or `head_mask`. Falling back to the manual attention implementation, but specifying the manual implementation will be required from Transformers version v5.0.0 onwards. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.


OutOfMemoryError: CUDA out of memory. Tried to allocate 624.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 329.06 MiB is free. Process 4665 has 14.42 GiB memory in use. Of the allocated memory 13.88 GiB is allocated by PyTorch, and 429.11 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
,